In [18]:
import pandas as pd
import numpy as np
import math
import random

import matplotlib.pyplot as plt
import seaborn as sns
import folium 


from kaggle.api.kaggle_api_extended import KaggleApi

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path


%matplotlib inline

## 1. Загрузка датасета 

In [2]:
!kaggle datasets list -s 'chipotle'

ref                                    title                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------  -----------------------------  -----  -------------------  -------------  ---------  ---------------  
jeffreybraun/chipotle-locations        Chipotle Locations             124KB  2020-07-28 20:20:41           2942         74  1.0              
andyxie/chipotle                       chipotle                        35KB  2017-12-03 23:34:39            359          4  0.1764706        
navneethc/chipotle                     Chipotle                        35KB  2017-11-07 21:37:54            985          9  0.23529412       
jongkilee/adp2020chipotleonlineretail  adp2020-chipotle-onlineretail   35KB  2020-11-26 10:43:54              7          1  0.11764706       


In [3]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files("jeffreybraun/chipotle-locations", path=Path.cwd(), unzip=True)

In [4]:
df = pd.read_csv('chipotle_stores.csv')

In [5]:
df.head()

,state,location,address,latitude,longitude
0,Alabama,Auburn,"346 W Magnolia Ave Auburn, AL 36832 US",32.606813,-85.487328
1,Alabama,Birmingham,"300 20th St S Birmingham, AL 35233 US",33.509721,-86.802756
2,Alabama,Birmingham,"3220 Morrow Rd Birmingham, AL 35235 US",33.595581,-86.647437
3,Alabama,Birmingham,"4719 Highway 280 Birmingham, AL 35242 US",33.422582,-86.698279
4,Alabama,Cullman,"1821 Cherokee Ave SW Cullman, AL 35055 US",34.154134,-86.841220


# 2. Описание датасета

В датасете содержится 2 629 строк и 5 столбцов:

- **state:** штат.
- **location:** город.
- **address:** адрес.
- **latitude:** широта.
- **longitude:**  долгота. 

# 3. Отрисуем торговые точки на карте США

In [6]:
usa_map = folium.Map([39.358, -98.118], zoom_start=4)

for lat, lon, loc in zip(df.latitude, df.longitude, df.location):
    folium.CircleMarker([lat, lon],radius=7, color=None,
                        fill_color='red', fill_opacity=0.4,
                        tooltip="Location : "+str(loc)).add_to(usa_map)
    
    
usa_map

# 4. Выберем город с наибольшим количеством ресторанов

In [8]:
larg_num_outlets_city =  df['location'].value_counts().head(1).index[0]

larg_num_outlets_city

'New York'

In [10]:
ny_df = df[df.location.eq(larg_num_outlets_city)]

ny_df.head(3)

ny_df.shape

(52, 5)

# 5. Задача

Рассчитать расстояние до ближайшего ресторана от случайной точки на карте 

## 5.1 Преобразуем координаты в Декартову систему координат

In [11]:
# Определяем центральную широту и долготу

central_lat = (ny_df['latitude'].min() + ny_df['latitude'].max()) / 2.0
central_long = (ny_df['longitude'].min() + ny_df['longitude'].max()) / 2.0

EARTH_RADIUS = 6371. * 10**3

central_lat
central_long
EARTH_RADIUS 

6371000.0

In [12]:
def convert_to_y(lat):
    return (EARTH_RADIUS * (lat - central_lat)) * (math.pi / 180.)

def convert_to_x(long):
    return (EARTH_RADIUS * (long - central_long)) * (math.pi / 180.)

In [13]:
decart_ny_df = ny_df\
               .transform({'latitude': convert_to_y, 'longitude': convert_to_x})\
               .rename({'latitude':'y', 'longitude':'x'}, axis=1) 

decart_ny_df.head(3)

,y,x
1646,-7754.434623,-3752.379535
1647,-7224.034428,-3215.102768
1648,-1015.588459,1332.147658


## 5.2 Генерируем случайные точки на карте и ищем расстояние до ближайшего ресторана

In [25]:
def random_point():
    rand_x = random.uniform(decart_ny_df['x'].min(), decart_ny_df['x'].max())
    rand_y = random.uniform(decart_ny_df['y'].min(), decart_ny_df['y'].max())
    return rand_x, rand_y
    

def find_smallest_distance(point, df):
    min_dist = math.inf
    for index, row in df.iterrows():
        # формула Евклидового расстояния
        dist = math.sqrt((point[0] - row['x'])**2 + (point[1] - row['y'])**2)
        if dist < min_dist:
            min_dist = dist
    return min_dist

## 5.3 Подсчитываем вероятности событий (частотное определение вероятноти)

- Событие A: расстояние оказалось от 0 до 100 метров
- Событие B: расстояние оказалось от 100 до 1000 метров
- Событие C: расстояние оказалось от 1000 метров и больше

In [45]:
random.seed(56)
N = 1000
a_counter = 0
b_counter = 0
c_counter = 0

for i in range(N):
    rand_xy = random_point()
    dist = find_smallest_distance(rand_xy, decart_ny_df)
    if dist <= 100:
        a_counter += 1
    elif dist <= 1000:
        b_counter += 1
    else:
        c_counter += 1
    

In [48]:
P_A = a_counter / N
P_B = b_counter / N
P_C = c_counter / N

result_df = pd.DataFrame([[P_A, P_B, P_C]], columns=['Вероятность А', 'Вероятность B', 'Вероятность C'])

result_df

,Вероятность А,Вероятность B,Вероятность C
0,0.01,0.325,0.665
